## Hello world OpenCL example

In [1]:
%%file hello.cl
__kernel void hello() {
    int gid = get_global_id(0);
    printf("Hello world! I'm a thread in block %d\n", gid);
}

Overwriting hello.cl


In [2]:
 %%file hello_CL.c
#include <CL/cl.h>

#include <stdio.h>
#include <stdlib.h>

#define MAX_SOURCE_SIZE (0x100000)

#define GLOBAl_SIZE 4
#define LOCAL_SIZE 1

int main(int argc, char ** argv) {

	// Load kernel from file hello.cl

	FILE *kernelFile;
	char *kernelSource;
	size_t kernelSize;

	kernelFile = fopen("hello.cl", "r");

	if (!kernelFile) {

		fprintf(stderr, "No file named hello.cl was found\n");

		exit(-1);

	}
	kernelSource = (char*)malloc(MAX_SOURCE_SIZE);
	kernelSize = fread(kernelSource, 1, MAX_SOURCE_SIZE, kernelFile);
	fclose(kernelFile);

	// Getting platform and device information
	cl_platform_id platformId = NULL;
	cl_device_id deviceID = NULL;
	cl_uint retNumDevices;
	cl_uint retNumPlatforms;
	cl_int ret = clGetPlatformIDs(1, &platformId, &retNumPlatforms);
	ret = clGetDeviceIDs(platformId, CL_DEVICE_TYPE_DEFAULT, 1, &deviceID, &retNumDevices);

	// Creating context.
	cl_context context = clCreateContext(NULL, 1, &deviceID, NULL, NULL,  &ret);

	// Creating command queue
	cl_command_queue commandQueue = clCreateCommandQueue(context, deviceID, 0, &ret);

	// Create program from kernel source
	cl_program program = clCreateProgramWithSource(context, 1, (const char **)&kernelSource, (const size_t *)&kernelSize, &ret);	

	// Build program
	ret = clBuildProgram(program, 1, &deviceID, NULL, NULL, NULL);

	// Create kernel
	cl_kernel kernel = clCreateKernel(program, "hello", &ret);

	// Execute the kernel
	size_t globalItemSize = GLOBAl_SIZE;
	size_t localItemSize = LOCAL_SIZE;
	ret = clEnqueueNDRangeKernel(commandQueue, kernel, 1, NULL, &globalItemSize, &localItemSize, 0, NULL, NULL);

  printf("That's all!\n");		

	// Clean up, release memory.
	ret = clFlush(commandQueue);
	ret = clFinish(commandQueue);
	ret = clReleaseCommandQueue(commandQueue);
	ret = clReleaseKernel(kernel);
	ret = clReleaseProgram(program);	
	ret = clReleaseContext(context);

	return 0;
}

Overwriting hello_CL.c


In [3]:
 !nvcc -o hello_CL hello_CL.c -lOpenCL && ./hello_CL

Hello world! I'm a thread in block 3
Hello world! I'm a thread in block 0
Hello world! I'm a thread in block 1
Hello world! I'm a thread in block 2
That's all!
